In [8]:
import pandas as pd
import sys
sys.path.append("/root/workSpace/investProject/ConvertBondWheel")

In [9]:
from policy.double_low_enhance.alarm import *

In [10]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO)

In [11]:
#对应配置 /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/config.py
double_low_conf = {
    'db_topk': 0.1,
    'min_curr_iss_amt': 9,  # <剩余规模
    'volatility': 0.1,  # >波动率
    'premium_rt': 0.2,  # <溢价率
    'min_price': 115,  # <可转债价格
    'min_dblow': 125,  # <双低价格
    'redeem': ['Y'],  #  Y=强赎 不在强赎
    'min_year_left': 1,  # >剩余到期年限 (不要一年内到期的)
    'ytm_rt': -0.01,  # > 到期收益率
    'pluse_out_step':10, # 脉冲轮出时， 轮出债比轮入新债双低值大多少
    'pulse_out_price_median':110, # 脉冲轮出时， 市场价格中位数
    'pulse_out_price':[120, 125], # 脉冲轮出时< 110 和>110 对应的轮出价格
    'pulse_out_dblow':[125, 130],  #同上， 双低值
    'user_zh_columns':[u'转债名称', u'双低', u'现价', u'溢价率', u'剩余规模', u'波动率', u'涨跌幅', u'成交额(万元)', u'换手率'],
} 

#user_zh_columns
# 展示字段， 可以继续添加，
# 如果报错， 说明没有爬取到该字段， 可以通过以下步骤来新增：
# 1. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/fields_config.py
# jisilu_dict 新增对应key-value, jisulu_used_columns 新增对应英文名
# 2. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/items.py
# 添加  该字段 = scrapy.Field()

In [33]:
# 读当前可转债数据
bound = read_data()
balance = read_balancing()

#轮动策略
in_bound, out_bound = periodWheel(bound,double_low_conf)

# 结果换成中文header & 按列排序
result = to_display(bound, balance, in_bound, out_bound, user_zh_columns=double_low_conf['user_zh_columns'])
result

,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*靖远转债,108.20,101.29,6.91%,28.00,4555.00%,0.29%,6086.34,215.00%,127027,靖远煤电,3.06,-0.97%,0.83,3.23,94.74,AA+,2.26,4.20,40.10%,2021-06-16,26-12-09,5.50,2.43%,1.92%,X
1,*文科转债,108.80,99.00,9.80%,9.50,2806.00%,0.09%,788.43,84.00%,128127,文科园林,4.40,0.69%,0.91,4.88,90.16,AA-,3.42,6.34,48.50%,2021-03-01,26-08-19,5.20,4.09%,3.34%,X
2,*鸿达转债,109.31,109.00,0.31%,7.78,4842.00%,0.02%,7486.75,884.00%,128085,鸿达兴业,4.26,0.24%,1.48,3.92,108.67,B,2.74,5.10,6.10%,2020-06-22,25-12-16,4.52,3.03%,2.10%,X
3,*东湖转债,110.66,104.03,6.63%,15.50,3036.00%,-0.19%,2877.71,179.00%,110080,东湖高新,6.01,-0.50%,0.88,6.16,97.56,AA,4.31,8.01,33.20%,2021-10-18,27-04-12,5.84,1.78%,1.30%,X
5,*孚日转债,111.54,102.50,9.04%,6.50,3187.00%,-0.55%,1145.88,172.00%,128087,孚日股份,4.23,-0.47%,1.06,4.50,94.00,AA,3.15,5.85,17.00%,2020-06-23,25-12-17,4.52,2.21%,1.67%,X
6,*利群转债,112.91,99.93,12.98%,18.00,2435.00%,-0.08%,510.34,28.00%,113033,利群股份,6.20,-0.16%,1.09,7.01,88.45,AA,4.91,9.11,34.80%,2020-10-09,26-03-31,4.81,3.00%,2.42%,X
7,*中装转2,113.05,101.32,11.73%,11.60,3970.00%,0.28%,3869.02,330.00%,127033,中装建设,5.74,0.35%,1.23,6.33,90.68,AA,4.43,8.23,31.70%,2021-10-22,27-04-15,5.85,2.55%,2.02%,X
9,*嘉泽转债,114.96,105.33,9.63%,11.84,4954.00%,0.32%,968.37,77.00%,113039,嘉泽新能,3.43,0.88%,1.96,3.57,96.08,AA,2.50,4.64,16.40%,2021-03-01,26-08-23,5.21,1.40%,0.94%,X
10,*英特转债,116.25,109.52,6.73%,6.00,6541.00%,-1.06%,2938.29,446.00%,127028,英特集团,13.75,-2.69%,1.89,13.40,102.61,AA+,9.38,17.42,21.00%,2021-07-12,27-01-04,5.57,0.59%,0.16%,X
12,*海波转债,117.17,109.70,7.47%,2.45,4074.00%,0.46%,462.01,173.00%,123080,海波重科,11.82,-0.34%,2.57,11.58,102.07,A+,8.11,15.05,18.40%,2021-06-08,26-12-01,5.48,2.70%,1.87%,X


In [ ]:
result.describe()

In [34]:
! sh /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/run_alarm.sh

---------2021-06-06 14:19:43------
---------scrapy---------
2021-06-06 14:19:44 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: ConvertBondWheel)
2021-06-06 14:19:44 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 07:30:14) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.1.1, Platform Linux-3.10.0-862.el7.x86_64-x86_64-with-glibc2.10
2021-06-06 14:19:44 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'ConvertBondWheel',
 'CONCURRENT_REQUESTS': 64,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 64,
 'CONCURRENT_REQUESTS_PER_IP': 64,
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'ConvertBondWheel.spiders',
 'SPIDER_MODULES': ['ConvertBondWheel.spiders']}
2021-06-06 14:19:44 [scrapy.extensions.telnet] INFO: Telnet Password: b298641efc1c1ea6
2021-06-06 14:19:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 '

--------alarm-----------
/root/workSpace/investProject/ConvertBondWheel
INFO:root:[/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py pulse start!]
DEBUG:root:Read bound data:
DEBUG:root:      dblow  bond_id bond_nm  ...  curr_iss_amt  redeem_flag turnover_rt
0    109.65   128127    文科转债  ...         9.498            X        1.87
1    110.39   127027    靖远转债  ...        28.000            X        1.51
2    110.80   110080    东湖转债  ...        15.500            X        1.20
3    111.49   128013    洪涛转债  ...        11.590            X        0.88
4    112.59   127033    中装转2  ...        11.600            X        3.11
..      ...      ...     ...  ...           ...          ...         ...
358  435.99   113580    康隆转债  ...         0.411            N      275.90
359  436.43   110044    广电转债  ...         0.709            N       33.99
360  482.36   123015    蓝盾转债  ...         1.003            X       58.23
361  516.00   128041    盛路转债  ...         0.530     